In [ ]:
# i will try saving everything in one file such that i dont have to open files

In [1]:
from architectures import *
from dataset_numpy import *
from utils import *
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import json
import h5py
from icecream import ic
import numpy as np

#getting the configurations from the config file
with open('working_config.json', 'r') as f:
    config = json.load(f)
    
testset = RandomSeqDataset(config["dataset_path"],config["testset_key"])
trainset = RandomSeqDataset(config["dataset_path"],config["trainset_key"])


# Create dataloaders from each subset
test_loader = DataLoader(testset,  # we want to load our dataset
                         shuffle=False,  # shuffle for training
                         batch_size=1,  # 1 sample at a time
                         num_workers=0,
                         collate_fn=stack_if_possible_collate_fn  # no background workers
                         )

training_loader = DataLoader(trainset,  # we want to load our dataset
                             shuffle=False,  # shuffle for training
                             batch_size=4,  # stack 4 samples to a minibatch
                             num_workers=0,
                             collate_fn=stack_if_possible_collate_fn  # 2 background workers
                             )

In [37]:
import torch.nn as nn
import torch
import numpy as np
import torch.nn.functional as F

class AE_CNN(nn.Module):
    def __init__(self, ae_input_dim = 90*90, ae_middle_dim = 80*80, ae_hidden_dim = 70*70,
                n_input_channels = 1, n_output_channels = 1, conv_kernel_size = 5, n_hidden_layers=5, n_hidden_kernels=32):
        super(AE_CNN, self).__init__()
        # encoder
        self.encoder_linear1 = nn.Linear(ae_input_dim, ae_middle_dim)
        self.encoder_linear2 = nn.Linear(ae_middle_dim, ae_hidden_dim)
        
        # convolution
        hidden_layers = []
        for _ in range(n_hidden_layers):
            # Add a CNN layer
            layer = nn.Conv2d(in_channels=n_input_channels,
                              out_channels=n_hidden_kernels,
                              kernel_size=conv_kernel_size)
            hidden_layers.append(layer)
            # Add relu activation module to list of modules
            hidden_layers.append(nn.ReLU())
            n_input_channels = n_hidden_kernels

        # deconvolution
        for _ in range(n_hidden_layers + 1):
            # Add a CNN layer
            layer = nn.ConvTranspose2d(in_channels=n_input_channels,
                                       out_channels=n_hidden_kernels,
                                       kernel_size=conv_kernel_size)
            hidden_layers.append(layer)
            # Add relu activation module to list of modules
            hidden_layers.append(nn.ReLU())
            n_input_channels = n_hidden_kernels

            
        self.hidden_layers = nn.Sequential(*hidden_layers)

        self.output_layer = nn.Conv2d(in_channels=n_input_channels,
                                      out_channels=n_output_channels,
                                      kernel_size=conv_kernel_size)

        
        # decoder
        self.decoder_linear1 = nn.Linear(ae_hidden_dim, ae_middle_dim)
        self.decoder_linear2 = nn.Linear(ae_middle_dim, ae_input_dim)

    def encoder(self, x):
        x = self.encoder_linear1(x)
        x = F.relu(x)
        x = self.encoder_linear2(x)
        x = F.relu(x)
        return x

    def decoder(self, x):
        x = self.decoder_linear1(x)
        x = F.relu(x)
        x = self.decoder_linear2(x)
        x = torch.sigmoid(x)
        return x
    
    #def create_outputs(self,x,known_array):
    #    output = []
    #    for i in range(4):
    #        output.append(x[i][known_array[i]==0])
    #    return output
    
    def create_outputs(self,x,known_array):
        return x[known_array==0]

    def forward(self, x, known_array):
        x = x.view(-1,1,90*90)
        x = self.encoder(x)
        x = x.view(-1,1,70,70)
        
         # Apply hidden layers module
        x = self.hidden_layers(x)

        # Apply last layer (=output layer)
        x = self.output_layer(x)
        x = x.view(-1,1,70*70)
        x = self.decoder(x)
        x = x.view(-1,1,90,90)
        x=F.sigmoid(x)
        return self.create_outputs(x,known_array)

net = AE_CNN()
#torch.save(net, os.path.join(config["results_path"], "train.pt"))
net

AE_CNN(
  (encoder_linear1): Linear(in_features=8100, out_features=6400, bias=True)
  (encoder_linear2): Linear(in_features=6400, out_features=4900, bias=True)
  (hidden_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (7): ReLU()
    (8): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (9): ReLU()
    (10): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (11): ReLU()
    (12): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (13): ReLU()
    (14): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (15): ReLU()
    (16): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (17): ReLU()
    (18): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (19): ReLU()
    (

In [38]:
# Create an instance of our NET
#net = torch.load(os.path.join("results", "train.pt"))

# GPU will be much faster here
device = torch.device(config["device"])
net.to(device=device)

# also try rmse and absolute error
loss_function = torch.nn.MSELoss(reduction="mean")

# Use a SGD optimizer
optimizer = torch.optim.SGD(net.parameters(), lr=1e-1)
#optimizer = torch.optim.Adam(net.parameters(), lr=config["learningrate"])

# Define a tensorboard summary writer that writes to directory "results_path/tensorboard"
writer = SummaryWriter(log_dir=os.path.join(config["results_path"], 'tensorboard'), flush_secs=1)

best_loss = np.float("inf")
# Optimize our dsnn model using SGD:
print("Continuing the training:")

# for update in tqdm(range(200)):
with tqdm(total=config["n_updates"]) as pbar:
    for update in range(config["n_updates"]):
        x = 0
        for data in training_loader:
            # Compute the output
            input_array, known_array, target_array = data
            #ic(target_array)
            # ic(input_array.unsqueeze(1).type(torch.float32).shape)
            #ic(input_array.unsqueeze(1).type(torch.float32).shape)
            output = net(input_array.unsqueeze(1).type(torch.float32),known_array.unsqueeze(1).type(torch.float32))
            #ic(output[300])
            # Compute the loss
            new_target = torch.tensor(np.concatenate(target_array, axis = None))
            #ic(output, target_array)
            #loss = 0
            #for target,out in zip(target_array,output):
                #loss+= loss_function(out, target)
            loss = loss_function(output.type(torch.float32), new_target.type(torch.float32))
            #loss = torch.mean((output-new_target)**2)
            ic(loss)
            # Compute the gradients
            loss.backward()
            # Preform the update
            optimizer.step()
            # Reset the accumulated gradients
            optimizer.zero_grad()
            x += 1
            if x == 1000:
                break

        # automatic saving of the best model
        if update % 1 == 0:
            with torch.no_grad():
                y = 0
                sum_loss = 0
                for tdata in test_loader:
                    input_array, known_array, target_array = data
                    output = net(input_array.unsqueeze(1).type(torch.float32),known_array.unsqueeze(1).type(torch.float32))
                    new_target = torch.tensor(np.concatenate(target_array, axis = None))
                    #ic(output.shape, new_target.shape)
                    sum_loss += loss_function(output, new_target.type(torch.float32))
                    y += 1
                    if y == 1:
                        ic(sum_loss)
                        break
                    
            #if sum_loss < best_loss:
            #  #  torch.save(net, os.path.join(config["results_path"], "autosave", f"loss_{sum_loss}.pt"))
            #    best_loss = loss

        # Tensorboard
        #if update % 1 == 0:
        #    # Add losse as scalars to tensorboard
        #    writer.add_scalar(tag="training/loss", scalar_value=sum_loss,
        #                      global_step=update)
#
        #    # Add images to Tensorboard
        #    writer.add_image(tag="training/output", img_tensor=output[0], global_step=update)
#
        #    writer.add_image(tag="training/input", img_tensor=(original.unsqueeze(1)[0]), global_step=update)

        torch.save(net, os.path.join(config["results_path"], "train.pt"))
        pbar.update(1)

print("training finished")
#torch.save(net, os.path.join(config["results_path"], "my_model_A.pt"))
print("model saved")

  0%|          | 0/100 [00:00<?, ?it/s]

Continuing the training:


C:\Users\fabio\.conda\envs\programing_in_python_2\lib\site-packages\torch\nn\functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
ic| loss: tensor(0.0695, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0671, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0837, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0515, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0783, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0644, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0888, grad_fn=<MseLossBackward>)
  0%|          | 0/100 [00:06<?, ?it/s]


KeyboardInterrupt: 

In [33]:
#predictions
#open the testfile
import pickle
import os
with open(os.path.join('data','testset.pkl'), 'rb') as pickle_file:
    arrays = pickle.load(pickle_file)
len(arrays['input_arrays'])

model = net
#creating the predictions
predictions_server = []
for i in range(len(arrays['input_arrays'])):
    inp = arrays['input_arrays'][i].reshape(1,90,90)
    known = arrays['known_arrays'][i].reshape(1,1,1,90,90)
    #ic(inp.shape,known.shape)
    out = model(torch.tensor(inp,dtype = torch.float32)/255,known)
    #ic(out.shape)
    pred=out.detach().numpy()
    predictions_server.append(np.array(pred*255,dtype=np.uint8 ))
    
with open(os.path.join('data','testset_submission_does_not_train_ae_convolution.pkl'), 'wb') as pickle_file:
    pickle.dump(predictions_server, pickle_file)

In [34]:
#open the testfile
import pickle
import os
with open(os.path.join('data','testset_submission_does_not_train_ae_convolution.pkl'), 'rb') as pickle_file:
    convolution = pickle.load(pickle_file)
convolution

[array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 126, 127, ..., 127, 127, 127], dtype=uint8),
 array([127, 1

In [34]:
1()

<>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<ipython-input-34-493eabe09421>:1: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  1()


TypeError: 'int' object is not callable

In [3]:
# also try rmse and absolute error
loss_function = torch.nn.MSELoss(reduction="mean")

# Use a SGD optimizer
optimizer = torch.optim.SGD(net.parameters(), lr=1e-7)

for data in training_loader:
    input_array, known_array, target_array = data
    #ic(target_array)
    # ic(input_array.unsqueeze(1).type(torch.float32).shape)
    #ic(input_array.unsqueeze(1).type(torch.float32).shape)
    output = net(input_array.unsqueeze(1).type(torch.float32),known_array.unsqueeze(1).type(torch.float32))
    
    # Compute the loss
    new_target = torch.tensor(np.concatenate(target_array, axis = None))
    #ic(output.shape, new_target.shape)
    loss = loss_function(output, new_target.type(torch.float32))
    ic(loss)
    # Compute the gradients
    loss.backward()
    # Preform the update
    optimizer.step()
    # Reset the accumulated gradients
    optimizer.zero_grad()
   

ic| loss: tensor(0.0486, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0564, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0605, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0427, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0341, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0420, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.1100, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0360, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0832, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0861, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0656, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0972, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0687, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0692, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0279, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0344, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0381, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0476, grad_fn=<MseLossBackward>)
ic| loss: tensor(0.0693, grad_fn=<MseLossBackward>)
ic| loss: te

TypeError: The first input argument needs to be a sequence

In [40]:
x=0
for array in target_array:
    for element in array:
        x+=1
x

9137

In [46]:
y = 0
for element in output:
    for e in element:
        y+=1
y

2171

In [55]:
x = 7
(x*90*2+x*(90-2*x)*2)*4

9296

In [53]:
x = 9
90*90-(90-2*x)**2

2916

In [36]:
for e in output:
    print(e.shape)

torch.Size([2171])


In [54]:
x = torch.tensor(np.random.randint(0,9,(4,1,5,5)))
x

tensor([[[[2, 1, 3, 4, 0],
          [2, 6, 2, 3, 1],
          [6, 3, 2, 8, 0],
          [5, 2, 7, 3, 4],
          [5, 2, 0, 0, 1]]],


        [[[2, 4, 4, 2, 0],
          [5, 4, 5, 2, 4],
          [3, 6, 4, 6, 0],
          [1, 3, 2, 0, 4],
          [5, 1, 4, 6, 6]]],


        [[[0, 5, 5, 2, 5],
          [4, 0, 0, 7, 8],
          [0, 8, 5, 8, 5],
          [2, 2, 4, 6, 4],
          [7, 0, 3, 3, 8]]],


        [[[3, 7, 2, 7, 3],
          [7, 7, 5, 1, 2],
          [1, 7, 2, 0, 5],
          [5, 6, 3, 7, 6],
          [8, 2, 4, 8, 7]]]], dtype=torch.int32)

In [55]:
y = torch.tensor(np.random.randint(0,2,(4,1,5,5)))
y

tensor([[[[1, 1, 0, 1, 0],
          [1, 0, 1, 1, 1],
          [1, 0, 1, 1, 0],
          [0, 1, 0, 0, 0],
          [0, 1, 0, 0, 1]]],


        [[[0, 0, 0, 1, 0],
          [1, 1, 0, 0, 0],
          [1, 1, 0, 0, 1],
          [0, 0, 0, 1, 1],
          [1, 1, 0, 0, 1]]],


        [[[0, 0, 0, 1, 0],
          [0, 0, 0, 1, 1],
          [1, 1, 1, 1, 1],
          [0, 1, 1, 0, 0],
          [1, 0, 0, 1, 1]]],


        [[[1, 0, 1, 1, 0],
          [1, 1, 0, 1, 0],
          [1, 1, 0, 1, 0],
          [0, 1, 1, 1, 0],
          [0, 0, 0, 1, 1]]]], dtype=torch.int32)

In [104]:
output = []
for i in range(4):
    output.append(x[i][y[i]==0])
output

[tensor([3, 0, 6, 3, 0, 5, 7, 3, 4, 5, 0, 0], dtype=torch.int32),
 tensor([2, 4, 4, 0, 5, 2, 4, 4, 6, 1, 3, 2, 4, 6], dtype=torch.int32),
 tensor([0, 5, 5, 5, 4, 0, 0, 2, 6, 4, 0, 3], dtype=torch.int32),
 tensor([7, 3, 5, 2, 2, 5, 5, 6, 8, 2, 4], dtype=torch.int32)]

In [102]:
mask0 = x[0][y[0]==0]
mask1 = x[1][y[1]==0]
mask2 = x[2][y[2]==0]
mask3 = x[3][y[3]==0]
mask = [mask0,mask1,mask2,mask3]

ValueError: only one element tensors can be converted to Python scalars

In [97]:
x[:,0].shape

torch.Size([4, 5, 5])

In [41]:
net

AE_CNN(
  (encoder_linear1): Linear(in_features=8100, out_features=6400, bias=True)
  (encoder_linear2): Linear(in_features=6400, out_features=4900, bias=True)
  (hidden_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (3): ReLU()
    (4): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (5): ReLU()
    (6): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (7): ReLU()
    (8): Conv2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (9): ReLU()
    (10): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (11): ReLU()
    (12): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (13): ReLU()
    (14): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (15): ReLU()
    (16): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (17): ReLU()
    (18): ConvTranspose2d(32, 32, kernel_size=(5, 5), stride=(1, 1))
    (19): ReLU()
    (

net = AE_CNN()
torch.save(net, os.path.join(config["results_path"], "AE_CNN_model1.pt"))

In [9]:
# Create an instance of our NET
net = torch.load(os.path.join("results", "my_model_A.pt"))

# GPU will be much faster here
device = torch.device(config["device"])
net.to(device=device)

# also try rmse and absolute error
loss_function = torch.nn.MSELoss(reduction="mean")

# Use a SGD optimizer
optimizer = torch.optim.SGD(net.parameters(), lr=1e-2)
#optimizer = torch.optim.Adam(net.parameters(), lr=config["learningrate"])

# Define a tensorboard summary writer that writes to directory "results_path/tensorboard"
writer = SummaryWriter(log_dir=os.path.join(config["results_path"], 'tensorboard'), flush_secs=1)

best_loss = np.float("inf")
# Optimize our dsnn model using SGD:
print("Continuing the training:")

# for update in tqdm(range(200)):
with tqdm(total=config["n_updates"]) as pbar:
    for update in range(config["n_updates"]):
        x = 0
        for data in training_loader:
            # Compute the output
            input_array, known_array, target = data
            # ic(input_array.unsqueeze(1).type(torch.float32).shape)
            output = net(input_array.unsqueeze(1).type(torch.float32))

            # Compute the loss
            ic(output[0])
            loss = loss_function(output, target.type(torch.float32).unsqueeze(1))
            # Compute the gradients
            loss.backward()
            # Preform the update
            optimizer.step()
            # Reset the accumulated gradients
            optimizer.zero_grad()
            x += 1
            if x == 10:
                break

        # automatic saving of the best model
        if update % 1 == 0:
            with torch.no_grad():
                y = 0
                sum_loss = 0
                for tdata in test_loader:
                    input_array, known_array, target_array = data
                    output = net(input_array.unsqueeze(1).type(torch.float32))
                    sum_loss += loss_function(output, original.type(torch.float32).unsqueeze(1))
                    y += 1
                    if y == 10:
                        ic(sum_loss)
                        break
                    
            #if sum_loss < best_loss:
            #  #  torch.save(net, os.path.join(config["results_path"], "autosave", f"loss_{sum_loss}.pt"))
            #    best_loss = loss

        # Tensorboard
        #if update % 1 == 0:
        #    # Add losse as scalars to tensorboard
        #    writer.add_scalar(tag="training/loss", scalar_value=sum_loss,
        #                      global_step=update)
#
        #    # Add images to Tensorboard
        #    writer.add_image(tag="training/output", img_tensor=output[0], global_step=update)
#
        #    writer.add_image(tag="training/input", img_tensor=(original.unsqueeze(1)[0]), global_step=update)

        #torch.save(net, os.path.join(config["results_path"], "train.pt"))
        pbar.update(1)

print("training finished")
#torch.save(net, os.path.join(config["results_path"], "my_model_A.pt"))
print("model saved")

  0%|          | 0/100 [00:00<?, ?it/s]ic| output[0]: tensor([[[0.0157, 0.0150, 0.0188,  ..., 0.0149, 0.0121, 0.0138],
                        [0.0174, 0.0151, 0.0244,  ..., 0.0200, 0.0239, 0.0171],
                        [0.0127, 0.0126, 0.0144,  ..., 0.0102, 0.0191, 0.0141],
                        ...,
                        [0.0196, 0.0172, 0.0150,  ..., 0.0161, 0.0213, 0.0153],
                        [0.0231, 0.0172, 0.0292,  ..., 0.0288, 0.0119, 0.0120],
                        [0.0120, 0.0255, 0.0167,  ..., 0.0171, 0.0148, 0.0195]]],
                      grad_fn=<SelectBackward>)
  0%|          | 0/100 [00:00<?, ?it/s]

Continuing the training:


AttributeError: 'list' object has no attribute 'type'

110 sec with cpu

note to myself:
activate tensorboard like this:

activate programing_in_python_2
tensorboard --logdir=C:\Users\fabio\Google_Drive\AI_Studium\programing_python\ass2\ex5\results\tensorboard

http://localhost:6006/


In [ ]:
len(training_loader)/100*112/60

In [2]:
from PIL import Image
import numpy as np

def array_to_img(image):
    img = Image.fromarray(image,)
    img.show()

In [20]:
for data in test_loader:
    image,_ = data
    array_to_img(np.array(image)[0].reshape(90,90)*255)
    array_to_img(np.array(_)[0].reshape(90,90)*255)
    break

In [16]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.
